# Who Were The Nuns Analysis

Set up the program by importing the required modules and setting the file location

In [16]:
import os # To navigate the OS and read/save files
import re # Regular Expressions - to search for patterns in text
import datetime as dt
import pandas as pd 
import numpy as np

wwtnImport = 'C:\GitHub\WhoWereTheNuns\WWTNDatabase'

## Load data
For each of the files, load it into a dataframe of the same name.

In [17]:
def loadWWTN(folder,file):
    fileName = os.path.join(folder,file) + '.csv'
    df = pd.read_csv(fileName)
    return df

associated_people= loadWWTN(wwtnImport, 'associated_people')
convent= loadWWTN(wwtnImport, 'convent')
convents= loadWWTN(wwtnImport, 'convents')
creative_works= loadWWTN(wwtnImport, 'creative_works')
link_types= loadWWTN(wwtnImport, 'link_types')
linkages= loadWWTN(wwtnImport, 'linkages')    
locations= loadWWTN(wwtnImport, 'locations')    
notes= loadWWTN(wwtnImport, 'notes')   
nuns= loadWWTN(wwtnImport, 'nuns')
office= loadWWTN(wwtnImport, 'office')    
offices= loadWWTN(wwtnImport, 'offices')  
pdfs= loadWWTN(wwtnImport, 'pdfs')   
pdfxuids= loadWWTN(wwtnImport, 'pdfxuids')  
people= loadWWTN(wwtnImport, 'people')
people_places= loadWWTN(wwtnImport, 'people_places')   
places= loadWWTN(wwtnImport, 'places')    
professions= loadWWTN(wwtnImport, 'professions')    
quotes= loadWWTN(wwtnImport, 'quotes')    
relationships= loadWWTN(wwtnImport, 'relationships')    
religorders= loadWWTN(wwtnImport, 'religorders')    
srcdocs= loadWWTN(wwtnImport, 'srcdocs')    
srcrefs= loadWWTN(wwtnImport, 'srcrefs')    
srcxcids= loadWWTN(wwtnImport, 'srcxcids')    
srcxuids= loadWWTN(wwtnImport, 'srcxuids')    
variants= loadWWTN(wwtnImport, 'variants')

# Build a list of dates
Collate dates from each individual table into a singe central list
This allows the analysis and processing to be done in one place

## Nuns
Contains one record for each nun:
* date_birth Date of Birth
* date_death Date of death
Also captures the age at death

## Profession
Details of the nun professing at a convent. 
Not all nuns will have a record here (some did not profess)
Some will have multile records (they joined more than one order)
* Profession date
Also captures the age at profession

## Office
> There are some nuns recorded in the database, who were never members of an English convent – this is an anomaly

* Office - Date from
* Office - Date until

## Convent
Stays at each convent
There are some nuns who 
* Convent - Date from
* Convent - Date until
* Locations - Date from

In [18]:
# Nuns
eventDates = nuns[['uid','date_birth']].copy()
eventDates['event']= 'Birth'
eventDates = eventDates.append(nuns[['uid','date_death','age_death']])
eventDates['event'].fillna('Death', inplace = True)

# Professions
eventDates =eventDates.append(professions[['uid','ordid','date_profession','age_profession']])
eventDates["event"].fillna('Profession', inplace = True)

# Office
eventDates =eventDates.append(office[['uid','cid','officeid','date_from']])
eventDates.rename(columns = {'date_from':'date_office_from'}, inplace = True)
eventDates["event"].fillna('Took office', inplace = True)

eventDates =eventDates.append(office[['uid','cid','officeid','date_until']])
eventDates.rename(columns = {'date_until':'date_office_until'}, inplace = True)
eventDates["event"].fillna('Left office', inplace = True)

# Convent
eventDates = eventDates.append(convent[['uid','cid','date_from','age_entry']])
eventDates.rename(columns = {'date_from':'date_convent_from'}, inplace = True)
eventDates["event"].fillna('Arrived at Convent', inplace = True)

eventDates = eventDates.append(convent[['uid','cid','date_left']])
eventDates.rename(columns = {'date_left':'date_left_convent'}, inplace = True)
eventDates["event"].fillna('Left Convent', inplace = True)

eventDates = eventDates.append(locations[['cid','date_from']])
eventDates.rename(columns = {'date_from':'date_location_from'}, inplace = True)
eventDates["event"].fillna('Convent Location from', inplace = True)

eventDates = eventDates.append(locations[['cid','date_to']])
eventDates.rename(columns = {'date_to':'date_location_to'}, inplace = True)
eventDates["event"].fillna('Convent Location to', inplace = True)

In [19]:
eventDates['eventDate'] = eventDates[['date_birth', 'date_death', 'date_profession','date_office_from','date_office_until','date_convent_from','date_left_convent','date_location_from','date_location_to']].bfill(axis=1).iloc[:, 0]
eventDates['eventAge'] = eventDates[['age_death','age_profession','age_entry']].bfill(axis=1).iloc[:, 0]

In [21]:
eventDates[['eventAge']]

,uid,date_birth,event,date_death,age_death,ordid,date_profession,age_profession,cid,officeid,date_office_from,date_office_until,date_convent_from,age_entry,date_left_convent,date_location_from,date_location_to,eventDate,eventAge
0,"""GB001""",1765,Birth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1765,NaN
1,"""GB002""",NaN,Birth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""GB003""",1736,Birth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1736,NaN
3,"""GB004""",NaN,Birth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""GB005""",NaN,Birth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,NaN,NaN,Convent Location to,NaN,NaN,NaN,NaN,NaN,"""25""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1795.0,1795.0,NaN
24,NaN,NaN,Convent Location to,NaN,NaN,NaN,NaN,NaN,"""26""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1794.0,1794.0,NaN
25,NaN,NaN,Convent Location to,NaN,NaN,NaN,NaN,NaN,"""30""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1795.0,1795.0,NaN
26,NaN,NaN,Convent Location to,NaN,NaN,NaN,NaN,NaN,"""34""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1838.0,1838.0,NaN
